In [63]:
# Imports
import random

import math

class Vertex:
    def __init__(self, key, x, y):
        self.key = key
        self.x = x
        self.y = y
        self.d = float('-inf')  # Distance from source
        self.h = None  # Heuristic distance to destination
        self.pi = None
        self.pi_set = set()
        self.priority = 0
        self.parent = None

    def __lt__(self, other):
        return self.priority < other.priority

    def __gt__(self, other):
        return self.priority > other.priority


class Graph:
    def __init__(self):
        self.vertices = {}
        self.adjacency_list = {}

    def add_vertex(self, key, x, y):
        if key not in self.vertices:
            self.vertices[key] = Vertex(key, x, y)
            self.adjacency_list[key] = []

    def add_edge(self, u, v):
        if u in self.vertices and v in self.vertices and u != v:
            if v not in self.adjacency_list[u]:
                self.adjacency_list[u].append(v)
            if u not in self.adjacency_list[v]:
                self.adjacency_list[v].append(u)  # For undirected graph


    def get_adjacency_list(self, vertex_key):
        if vertex_key in self.vertices:
            return self.adjacency_list[vertex_key]
        else:
            return []
    
    def max_degree(self):
        max_degree = 0
        for vertex_key in self.vertices.keys():
            degree = len(self.adjacency_list[vertex_key])
            max_degree = max(max_degree, degree)
        return max_degree
    
    def average_degree(self):
        total_degree = 0
        for vertex_key in self.vertices.keys():
            total_degree += len(self.adjacency_list[vertex_key])
        if len(self.vertices) > 0:
            return total_degree / len(self.vertices)
        else:
            return 0

    def generate_geometric_graph(self, n, r):

        for key in range(n):
            self.add_vertex(key,random.uniform(0, 1), random.uniform(0, 1))

        for u in self.vertices:
            for v in self.vertices:
                if u != v and euclidean_distance(self.vertices[u], self.vertices[v]) <= r:
                    self.adjacency_list[u].append(v)
                    self.adjacency_list[v].append(u)


def euclidean_distance(v1, v2):
    return math.sqrt(((v1.x - v2.x) ** 2) + ((v1.y - v2.y) ** 2))



n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

graph = Graph()

def largest_connected_component(graph):
    def dfs(graph, visited, vertex, component):
        visited.add(vertex)
        component.append(vertex)
        for neighbor in graph.get_adjacency_list(vertex):
            if neighbor not in visited:
                dfs(graph, visited, neighbor, component)

    visited = set()
    largest_component = []
    for vertex in graph.vertices:
        if vertex not in visited:
            component = []
            dfs(graph, visited, vertex, component)
            if len(component) > len(largest_component):
                largest_component = component

    lcc_graph = Graph()
    for vertex_key in largest_component:
        vertex = graph.vertices[vertex_key]
        lcc_graph.add_vertex(vertex.key, vertex.x, vertex.y)
        for neighbor_key in graph.get_adjacency_list(vertex_key):
            if neighbor_key in largest_component:
                lcc_graph.add_edge(vertex.key, neighbor_key)

    return lcc_graph


def binary_search_r(n, min_ratio, max_ratio):
    low = 0.0
    high = 1.0
    while low <= high:
        graph = Graph()
        mid = (low + high) / 2
        vlcc_min = int(min_ratio * n)
        vlcc_max = int(max_ratio * n)
        graph.generate_geometric_graph(n, mid)
        lcc = largest_connected_component(graph)
        lcc_size = len(lcc.vertices)
        if vlcc_min <= lcc_size <= vlcc_max:
            return mid, lcc
        elif lcc_size > vlcc_min:
            high = mid - 0.00001
        else:
            low = mid + 0.00001
    return None, None  # No suitable r found


def store_graph_to_file(graph, filename):
    with open(filename, 'w') as f:
        processed_edges = set()  # To avoid duplicates
        for u in graph.vertices:
            for v in graph.get_adjacency_list(u):
                # Generate a unique key for the edge (u, v) by sorting the vertices' keys
                edge_key = (u, v) if u < v else (v, u)
                if edge_key not in processed_edges:
                    vertex_u = graph.vertices[u]
                    vertex_v = graph.vertices[v]
                    f.write(f"{u} {vertex_u.x} {vertex_u.y} {v} {vertex_v.x} {vertex_v.y}\n")
                    processed_edges.add(edge_key)


n_values = [300, 400, 500]
min_ratios = [0.9, 0.8, 0.7]
max_ratios = [0.95, 0.9, 0.8]

# for n, min_ratio, max_ratio in zip(n_values, min_ratios, max_ratios):
#     r, lcc = binary_search_r(n, min_ratio, max_ratio)
#     if r is not None:
#         store_graph_to_file(lcc, f"graph_n{n}_r{r}.EDGES")
#         print(f"Graph with n={n} and r={r} stored successfully.")
#     else:
#         print(f"No suitable r found for n={n}.")


def read_graph_from_file(filename):
    graph = Graph()
    with open(filename, 'r') as f:
        for line in f:
            data = line.split()
            u = int(data[0])
            x1 = float(data[1])
            y1 = float(data[2])
            v = int(data[3])
            x2 = float(data[4])
            y2 = float(data[5])

            if u not in graph.vertices:
                graph.add_vertex(u, x1, y1)
            if v not in graph.vertices:
                graph.add_vertex(v, x2, y2)

            graph.add_edge(u, v)

    return graph


# graph = read_graph_from_file("graph_n300_r0.08202289062500001.EDGES")
graph = read_graph_from_file("graph_n400_r0.06249125.EDGES")
# graph = read_graph_from_file("graph_n500_r0.05467859375.EDGES")

if 13 in graph.vertices:
    print("yes")


yes


In [64]:
# DFS

def DFSAlgo(graph):
    def dfs(node, visited, path, memo):
        nonlocal max_length, longest_path
        visited[node] = True
        path.append(node)

        for neighbor in graph.adjacency_list[node]:
            if not visited[neighbor]:
                if neighbor not in memo or len(memo[neighbor]) < len(path):
                    dfs(neighbor, visited, path, memo)

        if len(path) > max_length:
            max_length = len(path)
            longest_path = path.copy()

        path.pop()
        visited[node] = False
        memo[node] = path.copy()

    max_length = float('-inf')
    longest_path = []

    visited = {v: False for v in graph.vertices}
    memo = {}

    for vertex in graph.vertices:
        if not visited[vertex]:
            dfs(vertex, visited, [], memo)

    return longest_path

a = DFSAlgo(graph)

print(a)
print(len(a))


[305, 269, 161, 179, 38, 223, 399, 26, 210, 323, 390, 155, 169, 348, 218, 189, 0, 322, 45, 94, 253, 247, 4, 67, 2, 89, 142, 198, 316, 69, 59, 71, 153, 158, 175, 315, 202, 137, 36, 314, 358, 19, 126, 102, 277, 241, 50, 53, 162, 58, 10, 27, 32, 265, 229, 55, 65, 47, 20, 11, 124, 257, 347, 88, 107, 68, 57, 74, 177, 335, 354, 40, 85, 92, 381, 237, 101, 127, 99, 77, 225, 160, 13, 29, 33, 39, 97, 64, 150, 172, 192, 220, 73, 123, 222, 246, 309, 296, 340, 294, 379, 186, 113, 62, 7, 46, 63, 163, 80, 51, 37, 297, 111, 307, 255, 215, 280, 333, 109, 81, 374]
121


In [65]:
# BFS

def longest_path_dp(graph):
    # Initialize a dictionary to store the length of the longest path ending at each vertex
    longest_paths = {vertex_key: [] for vertex_key in graph.vertices.keys()}
    visited = set()

    # Iterate over each edge in the graph
    for u in graph.vertices.keys():
        if u not in visited:
            stack = [(u, [u])]
            while stack:
                current_vertex_key, path = stack.pop()
                visited.add(current_vertex_key)
                for neighbor_key in graph.get_adjacency_list(current_vertex_key):
                    if neighbor_key not in visited:
                        new_path = path + [neighbor_key]
                        if len(new_path) > len(longest_paths[current_vertex_key]):
                            longest_paths[current_vertex_key] = new_path
                        stack.append((neighbor_key, new_path))

    # Find the vertex with the maximum length of the longest path
    end_vertex_key = max(longest_paths, key=lambda x: len(longest_paths[x]))

    return longest_paths[end_vertex_key]

# Example usage
longest_path = longest_path_dp(graph)
print("Longest path:", longest_path)
print(len(longest_path))

Longest path: [0, 322, 45, 369, 253, 247, 198, 382, 364, 227, 310, 342, 337, 209, 268, 230, 330, 298, 181, 6, 116, 175, 315, 202, 137, 314, 358, 19, 126, 277, 350, 53, 162, 78, 292, 166, 10, 263, 383, 299, 260, 303, 352, 270, 248, 211, 176, 146, 54, 207, 170, 47, 171, 238, 159, 141, 112, 339, 391, 11, 347, 107, 335, 354, 91, 85, 398, 381, 237, 204, 377, 319, 392, 340, 294, 379, 396, 285, 288, 235, 283, 119, 80, 200, 266, 51, 297, 37, 325, 306, 131, 105, 46, 387, 7, 63, 62, 113, 186, 143, 97, 172, 296, 309, 246, 338, 301, 222, 123, 73, 220, 192, 150, 64]
114


In [66]:
# Dijkstra

import heapq

def dijkstra_longest_path(graph, start):
    distances = {vertex: float('-inf') for vertex in graph.vertices}
    distances[start] = 0
    previous = {vertex: None for vertex in graph.vertices}

    pq = [(0, start)]  # Priority queue (distance, vertex)
    visited = set()

    while pq:
        distance, node = heapq.heappop(pq)

        if node in visited:
            continue

        visited.add(node)

        for neighbor in graph.adjacency_list[node]:
            if neighbor not in visited:
                new_distance = distances[node] + 1  # Assuming unweighted graph
                if new_distance > distances[neighbor]:
                    distances[neighbor] = new_distance
                    previous[neighbor] = node
                    heapq.heappush(pq, (-new_distance, neighbor))

    return distances, previous

def construct_path(previous, start, end):
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous[current]
    path.reverse()
    return path

def longest_path_dijkstra(graph):
    max_length = float('-inf')
    longest_path = []

    for vertex in graph.vertices:
        distances, previous = dijkstra_longest_path(graph, vertex)
        longest_distance = max(distances.values())
        if longest_distance > max_length:
            max_length = longest_distance
            end_vertex = max(distances, key=distances.get)
            longest_path = construct_path(previous, vertex, end_vertex)

    return longest_path, max_length

# Example usage:
# Replace Graph with your graph implementation
# graph = Graph()  # Assuming you have a graph instance
longest_path, length = longest_path_dijkstra(graph)
print(longest_path)
print(length)
# print("Length:", length)

[397, 38, 161, 179, 188, 210, 26, 12, 274, 384, 323, 390, 155, 169, 348, 218, 189, 0, 322, 45, 94, 253, 247, 4, 67, 2, 89, 142, 198, 316, 69, 59, 71, 153, 158, 175, 315, 202, 137, 36, 314, 358, 19, 126, 102, 277, 241, 50, 53, 162, 58, 10, 27, 32, 265, 229, 55, 65, 47, 20, 11, 124, 257, 347, 88, 107, 68, 57, 74, 177, 335, 354, 40, 85, 92, 381, 237, 101, 127, 99, 77, 225, 160, 13, 29, 33, 39, 97, 64, 150, 172, 192, 220, 73, 123, 222, 246, 309, 296, 340, 294, 379, 186, 113, 62, 7, 46, 63, 163, 80, 51, 37, 297, 111, 307, 255, 215, 280, 333, 109, 360, 343]
121


In [67]:
#  A star

import heapq

def initialize_single_source_max(graph, source, destination):
    for vertex in graph.vertices.values():
        vertex.d = float('-inf')
        vertex.h = euclidean_distance(vertex, destination)
        vertex.pi = None
        vertex.pi_set = set()

    source.d = 0

def relax_max(u, v):
    if v.d < (u.d + 1):
        v.d = u.d + 1
        v.pi = u
        return True
    return False

def a_star_max(graph, source, destination):
    initialize_single_source_max(graph, source, destination)
    closed_set = set()
    open_heap = []
    heapq.heappush(open_heap, (-source.d - source.h, source))
    
    while open_heap:
        _, u = heapq.heappop(open_heap)
        closed_set.add(u)

        if u == destination:
            break

        for v_key in graph.get_adjacency_list(u.key):
            v = graph.vertices[v_key]
            if v in closed_set:
                continue
            if relax_max(u, v):
                v.priority = v.d + v.h
                heapq.heappush(open_heap, (-v.priority, v))

final_path = []

for i in range(0, len(graph.vertices)):
    if i in graph.vertices:
        for j in range(0, len(graph.vertices)):
            if i != j and j in graph.vertices:
                s = graph.vertices[i]
                d = graph.vertices[j]
                a_star_max(graph, s, d)
                path = []
                x = d
                if x.d != float('-inf'):
                    visited = {x}
                    while x != s:
                        path.insert(0, x.key)
                        visited.add(x)
                        x = x.pi
                        if x in visited:
                            break
                    path.insert(0, s.key)
                    if (len(path) > len(final_path)):
                        final_path = path

final_path


print(final_path)
print(len(final_path))  